In [1]:
# use with local installs that don't have nbdev
import sys
sys.path.insert(0,"../../")

In [34]:
from selenium.webdriver.common.by import By
import datacrew.crawler.crawler as dcc
import datacrew.crawler.article as dca


URL = "https://domo-support.domo.com/s/article/360047400753?language=en_US"
URL = "https://domo-support.domo.com/s/article/360043429913"
BASE_URL = "https://domo-support.domo.com"

driver = dcc.driversetup(is_headless=False)

test_article = dca.Article_KB(url = URL, driver = driver, base_url = BASE_URL)

💤 loading https://domo-support.domo.com/s/article/360043429913 💤
Page https://domo-support.domo.com/s/article/360043429913 is loaded within 10 seconds.


In [26]:
import re

def convert_to_snake(text_str):
    """converts 'snake_case_str' to 'snakeCaseStr'"""

    return text_str.replace(" ", "_").lower()

def clean_url_name( path_name):
    valid_chars = r'[^a-zA-Z0-9_]'

    return re.sub(valid_chars, '', path_name)

In [41]:
# import textwrap
# from os import fdopen, remove
# from shutil import move, copymode
# from tempfile import mkstemp
# import mdutils
# import os
# from mdutils.mdutils import MdUtils
# import requests



def download_img(image_url, image_path):
    img_data = requests.get(image_url).content
    with open(image_path, 'wb') as handler:
        handler.write(img_data)


def output_md(
    article: dca.Article_KB, 
    output_folder: str = "../../raw_kb", 
    debug_prn: bool = False, 
):
    article_title = article.title
    
    output_path = os.path.join(output_folder, clean_url_name(convert_to_snake(article_title)))

    output_index = output_path

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    if debug_prn:
        print(f"outputing '{article_title}' to {output_path}")

    return output_path

        

test_article.get_images()
output_md(article = test_article, debug_prn= True)


outputing 'Adding a Beast Mode Calculation to Your Chart' to ../../raw_kb\adding_a_beast_mode_calculation_to_your_chart


'../../raw_kb\\adding_a_beast_mode_calculation_to_your_chart'

In [52]:
test_base_url = 'https://domo-support.domo.com/servlet/rtaImage'

def get_images(article, test_base_url: str = None, debug_prn: bool = False):
    
    image_ls = [{
           "url": f"{BASE_URL if item.get('src').startswith('/') else ''}{item.get('src')}",
            "relative_url": item.get('src'),
            "name": item.get('alt')} for item in article.soup.find_all('img')]

    if test_base_url:
        image_ls = [img for img in image_ls if img.get(
            'url').startswith(test_base_url)]

    
    return image_ls

get_images(test_article, test_base_url=test_base_url)


[{'url': 'https://domo-support.domo.com/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOEH',
  'relative_url': '/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOEH',
  'name': 'Beast_Mode_Dialog.jpg'},
 {'url': 'https://domo-support.domo.com/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOEE',
  'relative_url': '/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOEE',
  'name': ''},
 {'url': 'https://domo-support.domo.com/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOED',
  'relative_url': '/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOED',
  'name': 'Beast_Mode_Modal.png'},
 {'url': 'https://domo-support.domo.com/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOEE',
  'relative_url': '/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOEE',
  'name': 

In [ ]:


#     for img in article.image_ls:
#         download_img(image_url=img.get('url'),
#                      image_path=f"{output_path}\{img.get('name')}")

#         print(img.get('relative_url'))
#         article.md_str = article.md_str.replace(img.get('relative_url'), img.get('name'))

#     output_index = f"{output_path}\index"

# md_file = MdUtils(file_name=output_index)

# md_file.write(article.md_str)

# md_file.create_md_file()

# frontmatter_obj = f"""---
# title : {article.title}
# url : {article.url}
# linked_kbs :  { [ md_file.new_inline_link(link) for link in article.kb_url_ls]}
# article_id : {article.article_id}
# html : {article.kb_soup}
# views : {article.views}
# created_date : {str(article.created)}
# last updated : {str(article.last_updated)}
# ---"""

# print(f'front_matter {output_index}')
# add_frontmatter(data=frontmatter_obj, file_path=f"{output_index}.md")

# qmd_path = f"{output_index}.qmd"

# if os.path.exists(qmd_path):
#     os.remove(qmd_path)

# os.rename( f"{output_index}.md", qmd_path)


In [14]:
def dedent_frontmatter(file_path):
    # Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh, 'w') as new_file:
        with open(file_path) as old_file:
            count_frontmatter = 0
            for line in old_file:
                if count_frontmatter < 2:
                    if '---' in line:
                        count_frontmatter += 1
                    new_file.write(textwrap.dedent(line))
                else:
                    new_file.write(line)
    # Copy the file permissions from the old file to the new file
    copymode(file_path, abs_path)
    # Remove original file
    remove(file_path)
    # Move new file
    move(abs_path, file_path)

def add_frontmatter( data, file_path: str ):
    with open(file_path, 'r+', encoding='utf-8') as md_file:
        file_data = md_file.read()  # Save all the file's content
        md_file.seek(0, 0)  # Place file pointer at the beginning     
        md_file.write(data)   
        md_file.write('\n' + file_data)

    dedent_frontmatter(file_path)
    


In [ ]:
import logging

logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)


class Crawler:
    base_url: str
    output_folder: str
    urls_visited_ls: list[str]
    urls_to_vist_ls: list[str]

    driver: selenium.webdriver

    def __init__(self,
                 urls_to_visit_ls: list[str] = None,
                 base_url=None,
                 output_folder='../../raw_kb/',
                 ):

        self.base_url = base_url
        self.output_folder = output_folder
        self.urls_visited_ls = []
        self.urls_to_visit_ls = urls_to_visit_ls
        self.article_ls = []
        self.driver = dcc.driversetup(is_headless=False)

    def add_url_to_visit(self, url, debug_prn: bool = False):
        if url not in self.urls_visited_ls and url not in self.urls_to_visit_ls:
            if debug_prn:
                print(f'adding url to list - {url}')

            self.urls_to_visit_ls.append(url)

    def crawl(self, url, debug_prn: bool = False):
        if debug_prn:
            print(f"starting crawl - {url}")

        article = Article_KB(url=url, base_url=self.base_url, driver=driver)

        for url in article.kb_url_ls:
            self.add_url_to_visit(url=url, debug_prn=debug_prn)
        
        if article.is_success:
            output_md(article=article, output_folder=self.output_folder,
                    debug_prn=debug_prn)


    def run(self, debug_prn: bool = False):
        while self.urls_to_visit_ls:
            url = self.urls_to_visit_ls.pop(0)

            logging.info(f'Crawling: {url}')

            try:
                self.crawl(url, debug_prn)
            except Exception:
                logging.exception(f'Failed to crawl: {url}')
            finally:
                self.urls_visited_ls.append(url)

        print('done')
        return self


In [ ]:
crawler = Crawler(urls_to_visit_ls=[URL], base_url=BASE_URL, output_folder="../../raw_kb")

crawler.run(debug_prn=True)

2023-02-28 14:11:42,481 INFO:Crawling: https://domo-support.domo.com/s/article/360047400753?language=en_US


starting crawl - https://domo-support.domo.com/s/article/360047400753?language=en_US
💤 loading https://domo-support.domo.com/s/article/360047400753?language=en_US 💤


2023-02-28 14:11:46,753 INFO:Crawling: https://domo-support.domo.com/s/knowledge-base/


Page https://domo-support.domo.com/s/article/360047400753?language=en_US is loaded within 10 seconds.
adding url to list - https://domo-support.domo.com/s/knowledge-base/
adding url to list - https://domo-support.domo.com/s/
adding url to list - https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC
adding url to list - https://domo-support.domo.com/s/topic/0TO5w000000ZanzGAC
adding url to list - https://domo-support.domo.com/s/article/7872485267991
adding url to list - https://domo-support.domo.com/s/article/7440921035671
adding url to list - https://domo-support.domo.com/s/article/360047400753
adding url to list - https://domo-support.domo.com/s/topic/0TO5w000000ZanzGAC/other-connection-methods
adding url to list - https://domo-support.domo.com/s/article/360043429933
adding url to list - https://domo-support.domo.com/s/article/360042925494
adding url to list - https://domo-support.domo.com/s/article/360043429953
adding url to list - https://domo-support.domo.com/s/article/360043429

2023-02-28 14:12:04,300 INFO:Crawling: https://domo-support.domo.com/s/


...

2023-02-28 14:14:17,496 INFO:Crawling: https://domo-support.domo.com/s/knowledge-base/


Message: 
Stacktrace:
Backtrace:
	(No symbol) [0x00CB37D3]
	(No symbol) [0x00C48B81]
	(No symbol) [0x00B4B36D]
	(No symbol) [0x00B7D382]
	(No symbol) [0x00B7D4BB]
	(No symbol) [0x00BB3302]
	(No symbol) [0x00B9B464]
	(No symbol) [0x00BB1215]
	(No symbol) [0x00B9B216]
	(No symbol) [0x00B70D97]
	(No symbol) [0x00B7253D]
	GetHandleVerifier [0x00F2ABF2+2510930]
	GetHandleVerifier [0x00F58EC1+2700065]
	GetHandleVerifier [0x00F5C86C+2714828]
	GetHandleVerifier [0x00D63480+645344]
	(No symbol) [0x00C50FD2]
	(No symbol) [0x00C56C68]
	(No symbol) [0x00C56D4B]
	(No symbol) [0x00C60D6B]
	BaseThreadInitThunk [0x776900F9+25]
	RtlGetAppContainerNamedObjectPath [0x77A67BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77A67B8E+238]

Timeout Exception: Page https://domo-support.domo.com/s/login/ did not load within 10 seconds.
search term slds-form-element does not exist in https://domo-support.domo.com/s/login/
starting crawl - https://domo-support.domo.com/s/knowledge-base/
💤 loading https://domo-suppor

2023-02-28 14:14:33,876 INFO:Crawling: https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC


Message: 
Stacktrace:
Backtrace:
	(No symbol) [0x00CB37D3]
	(No symbol) [0x00C48B81]
	(No symbol) [0x00B4B36D]
	(No symbol) [0x00B7D382]
	(No symbol) [0x00B7D4BB]
	(No symbol) [0x00BB3302]
	(No symbol) [0x00B9B464]
	(No symbol) [0x00BB1215]
	(No symbol) [0x00B9B216]
	(No symbol) [0x00B70D97]
	(No symbol) [0x00B7253D]
	GetHandleVerifier [0x00F2ABF2+2510930]
	GetHandleVerifier [0x00F58EC1+2700065]
	GetHandleVerifier [0x00F5C86C+2714828]
	GetHandleVerifier [0x00D63480+645344]
	(No symbol) [0x00C50FD2]
	(No symbol) [0x00C56C68]
	(No symbol) [0x00C56D4B]
	(No symbol) [0x00C60D6B]
	BaseThreadInitThunk [0x776900F9+25]
	RtlGetAppContainerNamedObjectPath [0x77A67BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77A67B8E+238]

Timeout Exception: Page https://domo-support.domo.com/s/knowledge-base/ did not load within 10 seconds.
search term slds-form-element does not exist in https://domo-support.domo.com/s/knowledge-base/
starting crawl - https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC
💤 

KeyboardInterrupt: 